# Primeira Submissão de Teste

Nesta submissão, foi utilizado o modelo padrão da AlexNet, com os dados pré-treinados. Esse modelo foi treinado durante 100 épocas.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importando o fastai

In [2]:
from fastai.vision.all import *

## Inicialização do modelo pelo fastai

In [3]:
root_path = '/kaggle/input/cassava-leaf-disease-classification/'
df_train = pd.read_csv(root_path + 'train.csv')
dls = ImageDataLoaders.from_df(df_train, path=root_path+"train_images/", item_tfms=Resize(224))
learn = cnn_learner(dls, resnet152, metrics=[error_rate,accuracy], pretrained=False)

## Leitura do modelo treinado

In [4]:
model_path = '/kaggle/input/models-analise'
learn.model_dir = model_path
learn.load('resnet152_tune_10')

## Predição dos dados de teste

In [5]:
test_files = get_image_files('../input/cassava-leaf-disease-classification/test_images')
test_dl = learn.dls.test_dl(test_files, with_label=True)
pred_probas, x, pred_classes = learn.get_preds(dl=test_dl, with_decoded=True) 

## Gravação dos resultados no csv de submissão

In [6]:
rows = []
for file, label in zip(test_files, pred_classes):
    row = {'image_id': os.path.basename(file), 'label': int(label)}
    rows.append(row)
df = pd.DataFrame(rows)
df.to_csv('./submission.csv', index=False)